# Alphavantage Intraday
* API for "real time" stock data
* Freeium model - sign up fro free account to get API key
* Can make API calls directly or install the helper library:
pip install alpha_vantage from the command line

#### https://alphavantage.co

### 1. Set up Environment

In [ ]:
pip install alpha_vantage

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from alpha_vantage.timeseries import TimeSeries

### 2. Store API Key

In [ ]:
# replace with your own API key
key = "1Z8DLJEG0ISEJHI0"

### 3. Create TimeSeries Object and Make API Call

In [ ]:
ts = TimeSeries(key, output_format='pandas')
data, meta = ts.get_intraday('TSLA', interval='1min', outputsize='full')

### 4. Examine Data

In [ ]:
meta

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
plt.plot(data['4. close'])

### 5. Rename Columns and Add Separate Date and Time Columns

In [ ]:
columns = ['open', 'high', 'low', 'close', 'volume']
data.columns = columns

In [ ]:
data['TradeDate'] = data.index.date
data['time'] = data.index.time

In [ ]:
data.loc['2020-12-31']

In [ ]:
market = data.between_time('09:30:00', '16:00:00').copy()
market.sort_index(inplace=True)
market.info()

In [ ]:
market.groupby('TradeDate').agg({'low':min, 'high':max})

In [ ]:
market.loc[market.groupby('TradeDate')['low'].idxmin()]

In [ ]:
market.loc[market.groupby('TradeDate')['high'].idxmax()]

In [ ]:


def fetch_NSE_stock_price(stock_code):
    
    stock_url  = 'https://www.nseindia.com/live_market/dynaContent/live_watch/get_quote/GetQuote.jsp?symbol='+str(stock_code)
    #print(stock_url)
    headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}
    response = requests.get(stock_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    data_array = soup.find(id='responseDiv').getText().strip().split(":")
    
    for item in data_array:
        if 'lastPrice' in item:
            index = data_array.index(item)+1
            latestPrice=data_array[index].split('"')[1]
            lp =  float(latestPrice.replace(',',''))
        elif 'closePrice' in item:
            index = data_array.index(item)+1
            closePrice=data_array[index].split('"')[1]
            cp =  float(closePrice.replace(',',''))
        elif 'open' in item:
            index = data_array.index(item)+1
            openPrice=data_array[index].split('"')[1]
            op =  float(openPrice.replace(',',''))
        elif 'dayLow' in item:
            index = data_array.index(item)+1
            dayLow=data_array[index].split('"')[1]
            dl =  float(dayLow.replace(',',''))
        elif 'dayHigh' in item:
            index = data_array.index(item)+1
            dayHigh=data_array[index].split('"')[1]
            dh =  float(dayHigh.replace(',',''))
    return op,lp,dh, dl,cp       

nifty50_url = 'https://www1.nseindia.com/content/indices/ind_nifty50list.csv'
df_n50 = pd.read_csv(nifty50_url)

regexp = re.compile('&')

OP  = []
LP  = []
DHP = []
DLP = []
CP  = []


while True:
   try:
      for index, row in df_n50.iterrows():
         stock_code = row['Symbol']
         if(regexp.search(stock_code) != None):
            stock_code = stock_code.replace('&','%26')
         
         oPrice,lPrice,dhPrice, dlPrice,cPrice = fetch_NSE_stock_price(stock_code)
         OP.append(str(oPrice))
         LP.append(str(lPrice))
         DHP.append(str(dhPrice))
         DLP.append(str(dlPrice))
         CP.append(str(cPrice))

      os.system('cls')
      print("--------------------------------------------------------------------------------------------------------------------------------------------")
      print("|{:50s} | {:20s} | {:10s} | {:10s} | {:10s} | {:10s} | {:10s}|".format( 'Company Name','Symbol','openPrice','lastPrice','dayHigh','dayLow','closePrice'))
      print("--------------------------------------------------------------------------------------------------------------------------------------------")


      for index, row in df_n50.iterrows():
         stock_code = row['Symbol']
         
         print("|{:50s} | {:20s} | {:10s} |{:10s} | {:10s} | {:10s} | {:10s} |".format(str(row['Company Name']), row['Symbol'], OP[index].rjust(10), LP[index].rjust(10), DHP[index].rjust(10), DLP[index].rjust(10), CP[index].rjust(10)))
         
      print("--------------------------------------------------------------------------------------------------------------------------------------------")

   except KeyboardInterrupt:
        sys.exit()

In [ ]:
import pandas as pd
import sys
import requests
from bs4 import BeautifulSoup
import re
import os

In [ ]:

def fetch_NSE_stock_price(stock_code):

    try:
        stock_url  = 'https://www.nseindia.com/live_market/dynaContent/live_watch/get_quote/GetQuote.jsp?symbol='+str(stock_code)
        print("hi",stock_url)
        #     #print(stock_url)
        headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}
        response = requests.get(stock_url, headers=headers)
    except requests.exceptions.RequestException as e:  # This is the correct syntax
        raise SystemExit(e)

#     soup = BeautifulSoup(response.text, 'html.parser')
#     data_array = soup.find(id='responseDiv').getText().strip().split(":")
#     print(data_array)
    
#     for item in data_array:
#         if 'lastPrice' in item:
#             index = data_array.index(item)+1
#             latestPrice=data_array[index].split('"')[1]
#             lp =  float(latestPrice.replace(',',''))
#         elif 'closePrice' in item:
#             index = data_array.index(item)+1
#             closePrice=data_array[index].split('"')[1]
#             cp =  float(closePrice.replace(',',''))
#         elif 'open' in item:
#             index = data_array.index(item)+1
#             openPrice=data_array[index].split('"')[1]
#             op =  float(openPrice.replace(',',''))
#         elif 'dayLow' in item:
#             index = data_array.index(item)+1
#             dayLow=data_array[index].split('"')[1]
#             dl =  float(dayLow.replace(',',''))
#         elif 'dayHigh' in item:
#             index = data_array.index(item)+1
#             dayHigh=data_array[index].split('"')[1]
#             dh =  float(dayHigh.replace(',',''))
#     return op,lp,dh, dl,cp       

# nifty50_url = 'https://www1.nseindia.com/content/indices/ind_nifty50list.csv'
# df_n50 = pd.read_csv(nifty50_url)

# regexp = re.compile('&')

In [ ]:
fetch_NSE_stock_price('SBIN')

In [ ]:
stock_url  = 'https://www.nseindia.com/live_market/dynaContent/live_watch/get_quote/GetQuote.jsp?symbol=SBIM'
# response = requests.get(stock_url)


r = requests.get(stock_url, headers={
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36"
})

In [ ]:
import pandas as pd
import sys
import requests
from bs4 import BeautifulSoup
import re
import os

def fetch_NSE_stock_price(stock_code):
    
    stock_url  = 'https://www.nseindia.com/live_market/dynaContent/live_watch/get_quote/GetQuote.jsp?symbol='+str(stock_code)
    #print(stock_url)
    headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}
    response = requests.get(stock_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    data_array = soup.find(id='responseDiv').getText().strip().split(":")
    
    for item in data_array:
        if 'lastPrice' in item:
            index = data_array.index(item)+1
            latestPrice=data_array[index].split('"')[1]
            lp =  float(latestPrice.replace(',',''))
        elif 'closePrice' in item:
            index = data_array.index(item)+1
            closePrice=data_array[index].split('"')[1]
            cp =  float(closePrice.replace(',',''))
        elif 'open' in item:
            index = data_array.index(item)+1
            openPrice=data_array[index].split('"')[1]
            op =  float(openPrice.replace(',',''))
        elif 'dayLow' in item:
            index = data_array.index(item)+1
            dayLow=data_array[index].split('"')[1]
            dl =  float(dayLow.replace(',',''))
        elif 'dayHigh' in item:
            index = data_array.index(item)+1
            dayHigh=data_array[index].split('"')[1]
            dh =  float(dayHigh.replace(',',''))
    return op,lp,dh, dl,cp       

nifty50_url = 'https://www1.nseindia.com/content/indices/ind_nifty50list.csv'
df_n50 = pd.read_csv(nifty50_url)

regexp = re.compile('&')

OP  = []
LP  = []
DHP = []
DLP = []
CP  = []


while True:
   try:
      for index, row in df_n50.iterrows():
         stock_code = row['Symbol']
         if(regexp.search(stock_code) != None):
            stock_code = stock_code.replace('&','%26')
         
         oPrice,lPrice,dhPrice, dlPrice,cPrice = fetch_NSE_stock_price(stock_code)
         OP.append(str(oPrice))
         LP.append(str(lPrice))
         DHP.append(str(dhPrice))
         DLP.append(str(dlPrice))
         CP.append(str(cPrice))

      os.system('cls')
      print("--------------------------------------------------------------------------------------------------------------------------------------------")
      print("|{:50s} | {:20s} | {:10s} | {:10s} | {:10s} | {:10s} | {:10s}|".format( 'Company Name','Symbol','openPrice','lastPrice','dayHigh','dayLow','closePrice'))
      print("--------------------------------------------------------------------------------------------------------------------------------------------")


      for index, row in df_n50.iterrows():
         stock_code = row['Symbol']
         
         print("|{:50s} | {:20s} | {:10s} |{:10s} | {:10s} | {:10s} | {:10s} |".format(str(row['Company Name']), row['Symbol'], OP[index].rjust(10), LP[index].rjust(10), DHP[index].rjust(10), DLP[index].rjust(10), CP[index].rjust(10)))
         
      print("--------------------------------------------------------------------------------------------------------------------------------------------")

   except KeyboardInterrupt:
        sys.exit()

In [ ]:
# Raw Package
import numpy as np
import pandas as pd

#Data Source
import yfinance as yf

#Data viz
import plotly.graph_objs as go

#Interval required 1 minute
data = yf.download(tickers='UBER', period='1d', interval='1m')

#declare figure
fig = go.Figure()

#Candlestick
fig.add_trace(go.Candlestick(x=data.index,
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'], name = 'market data'))

# Add titles
fig.update_layout(
    title='Uber live share price evolution',
    yaxis_title='Stock Price (USD per Shares)')

# X-Axes
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=15, label="15m", step="minute", stepmode="backward"),
            dict(count=45, label="45m", step="minute", stepmode="backward"),
            dict(count=1, label="HTD", step="hour", stepmode="todate"),
            dict(count=3, label="3h", step="hour", stepmode="backward"),
            dict(step="all")
        ])
    )
)

#Show
fig.show()

In [ ]:
pip install nsepy

In [ ]:
from nsepy import get_history
from datetime import date
data = get_history(symbol="SBIN", start=date(2015,1,1), end=date(2015,1,31))
data[['Close']].plot()

In [ ]:
import pandas as pd
import pickle
URL = 'https://www1.nseindia.com/content/indices/ind_nifty50list.csv'
df = pd.read_csv(URL, index_col = 'Company Name')
df